In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
from common_import import *

root_3dhp = f'/home/{user}/Datasets/HAAI/3DHP'
source = 'original' # 
root_original = os.path.join(root_3dhp, source)
train_folder = os.path.join(root_original, 'train')
test_folder = os.path.join(root_original, 'test')

In [2]:
# Test video info
video_info = {'train': {}, 'test': {}}

# test
for subject in os.listdir(test_folder):
    if 'TS' not in subject: continue
    video_info['test'][subject] = {}
    num_frame = len(os.listdir(f'/home/hrai/Datasets/HAAI/3DHP/original/test/{subject}/imageSequence'))
    #img_list = natsorted(glob(f'/home/hrai/Datasets/HAAI/3DHP/original/test/{subject}/imageSequence/*.jpg'))
    img = get_img_frame_3dhp('test', subject, 1)
    H, W, C = img.shape
    video_info['test'][subject]['num_frame'] = num_frame
    video_info['test'][subject]['W'] = W
    video_info['test'][subject]['H'] = H
print(video_info['test'])

# train
os.listdir(train_folder)

{'TS2': {'num_frame': 6080, 'W': 2048, 'H': 2048}, 'TS5': {'num_frame': 320, 'W': 1920, 'H': 1080}, 'TS4': {'num_frame': 6009, 'W': 2048, 'H': 2048}, 'TS1': {'num_frame': 6151, 'W': 2048, 'H': 2048}, 'TS3': {'num_frame': 5839, 'W': 2048, 'H': 2048}, 'TS6': {'num_frame': 492, 'W': 1920, 'H': 1080}}


['S1', 'S6', 'S4', 'S3', 'S5', 'S8', 'S7', 'S2']

### Trainset

In [ ]:
# train camera parameters
f = open(os.path.join(train_folder, 'S1', 'Seq1', 'camera.calibration'), 'r')
lines = f.readlines()
name = -1
cam_dict = {}

for line in lines:
    item = line.strip().split(' ')
    item_type = item[0]
    #print(item)
    if item_type == 'name':
        name = f'cam{item[-1]}'
        cam_dict[name] = {}
    elif item_type == 'intrinsic':
        fx = float(item[3])
        cx = float(item[5])
        fy = float(item[8])
        cy = float(item[9])
        intrinsic = np.array([[fx, 0, cx], [0, fy, cy], [0, 0, 1]])
        cam_dict[name]['intrinsic'] = intrinsic
    elif item_type == 'extrinsic':
        R = np.array([[float(item[3]), float(item[4]), float(item[5])], [float(item[7]), float(item[8]), float(item[9]),], [float(item[11]), float(item[12]), float(item[13])]])
        t = np.array([float(item[6]), float(item[10]), float(item[14])])/1000
        C = -R.T@t
        #print(R.shape, t.shape)
        cam_dict[name]['R'] = R
        cam_dict[name]['t'] = t
        cam_dict[name]['C'] = C
        cam_dict[name]['extrinsic'] = np.hstack((R, t.reshape(-1, 1)))
    elif item_type == 'size':
        cam_dict[name]['W'] = float(item[-2])
        cam_dict[name]['H'] = float(item[-1])

savepkl(cam_dict, f'/home/{user}/codes/MotionBERT/custom_codes/dataset_generation/3dhp/3dhp_train_cam_params.pkl')

### Testset

In [3]:
cam_info_3dhp_test = get_3dhp_cam_info('test')

cam_dict = {}
for subject in ['TS1', 'TS2', 'TS3', 'TS4', 'TS5', 'TS6']:
    if subject in ['TS1', 'TS2', 'TS3', 'TS4']: info_key = 'test_cam_1_4'
    elif subject in ['TS5', 'TS6']: info_key = 'test_cam_5_6'

    R = cam_info_3dhp_test[info_key]['R']
    t = cam_info_3dhp_test[info_key]['t']
    C = cam_info_3dhp_test[info_key]['C']
    W = video_info['test'][subject]['W']
    H = video_info['test'][subject]['H']
    num_frames = video_info['test'][subject]['num_frame']

    extrinsic = np.hstack((R, t.reshape(-1, 1)))
    intrinsic = convert_intrinsic_from_mm_to_pixel(cam_info_3dhp_test[info_key]['sensor_size'],
                                                   cam_info_3dhp_test[info_key]['focal_length'],
                                                   cam_info_3dhp_test[info_key]['center_offset'],
                                                   W,
                                                   H)

    cam_param = {'R': R,
                 't': t,
                 'C': C,
                 'intrinsic': intrinsic,
                 'extrinsic': extrinsic,
                 'num_frames': num_frames,
                 'W': W,
                 'H': H}
    print(num_frames)
    cam_dict[subject] = cam_param

savepkl(cam_dict, f'/home/{user}/codes/MotionBERT/custom_codes/dataset_generation/3dhp/3dhp_test_cam_params.pkl')

R: [[ 5.75281000e-04 -2.08215000e-01  9.78083371e-01]
 [ 6.16098000e-02  9.76233000e-01  2.07784794e-01]
 [-9.98100000e-01  6.01400000e-02  1.33896928e-02]]
R: [[-0.93964726  0.02527235 -0.34121028]
 [-0.00921029  0.99503851  0.09906346]
 [ 0.34202093  0.09622737 -0.93475244]]
6151
6080
5839
6009
320
492


In [9]:
cam_dict = readpkl(f'/home/{user}/codes/MotionBERT/custom_codes/dataset_generation/3dhp/3dhp_test_cam_params.pkl')
data_dict, cam_params = load_3dhp_original('test')
data_dict.keys(), cam_params.keys()

(dict_keys(['TS1', 'TS2', 'TS3', 'TS4', 'TS5', 'TS6']),
 dict_keys(['TS1', 'TS2', 'TS3', 'TS4', 'TS5', 'TS6']))

### Check 

In [26]:
data_type_list = [
     'source_list', 'cam_param', 
     'cam_3d', 'img_2d',
]
overwrite_list = ['cam_param'] # ['cam_param', 'img_2d']
dataset_name = '3dhp'
data_aug = default_data_aug.copy()
data = load_data_dict(dataset_name, data_type_list, overwrite_list, verbose=True, data_aug=data_aug, univ=False)

4
[overwrite: False] ==> Loading 3DHP source_list...
4
[overwrite: True] ==> Loading 3DHP cam_param...


dict_keys(['cam0', 'cam1', 'cam2', 'cam3', 'cam4', 'cam5', 'cam6', 'cam7', 'cam8', 'cam9', 'cam10', 'cam11', 'cam12', 'cam13', 'TS1', 'TS2', 'TS3', 'TS4', 'TS5', 'TS6'])


100%|██████████| 230/230 [00:00<00:00, 234148.04it/s]

4797
6245
5015
5216
5532
5645
6184
5026
5175
5362
6266
5814
5530
4883
10865
11977
10830
10817
11266
12106
11847
10832
11648
11860
12037
11891
11428
9371
5395
5788
4162
4879
5345
6207
5702
4809
6206
6268
5943
6295
5959
4880
5623
5946
5681
5707
5335
5932
5917
3872
5929
5934
6081
5961
5750
5409
11711
11924
10904
11014
10674
11986
11906
9402
11880
12297
11494
12489
12391
9751
11823
11675
11064
11202
10374
11910
11685
8391
11977
12022
11426
12124
11988
9582
5775
6107
5417
5490
5678
6171
6051
5041
6143
6169
6107
6171
5847
5396
5916
6618
5835
5908
6195
6604
6418
5518
6580
6627
6594
6668
6220
5911
11520
12418
11093
11367
11380
12160
12293
10219
11727
12670
12259
12820
12423
10024
11922
11994
11450
11468
11612
12074
11956
10999
12215
12298
11904
12304
12150
10873
6082
6150
6019
6082
6058
6186
6119
5793
6188
6188
6188
6188
6142
5761
5805
6075
5901
5942
5860
6085
5971
5666
6081
6090
6145
6131
6030
5563
6040
6239
6038
6046
6098
6152
6206
5875
6083
6230
6202
6239
6128
5469
5832
6296
5859
5930
5964


In [8]:
cam_params

{'S1': {'Seq1': {'cam0': {'W': 2048.0,
    'H': 2048.0,
    'intrinsic': array([[1.497693e+03, 0.000000e+00, 1.024704e+03],
           [0.000000e+00, 1.497103e+03, 1.051394e+03],
           [0.000000e+00, 0.000000e+00, 1.000000e+00]]),
    'R': array([[ 0.9650164 ,  0.00488022,  0.262144  ],
           [-0.00448836, -0.9993728 ,  0.0351275 ],
           [ 0.262151  , -0.03507521, -0.9643893 ]]),
    't': array([-0.5628666,  1.398138 ,  3.852623 ]),
    'C': array([-0.46051813,  1.53513956,  3.81386741]),
    'extrinsic': array([[ 0.9650164 ,  0.00488022,  0.262144  , -0.5628666 ],
           [-0.00448836, -0.9993728 ,  0.0351275 ,  1.398138  ],
           [ 0.262151  , -0.03507521, -0.9643893 ,  3.852623  ]]),
    'num_frames': 15245755},
   'cam1': {'W': 2048.0,
    'H': 2048.0,
    'intrinsic': array([[1.495217e+03, 0.000000e+00, 1.030519e+03],
           [0.000000e+00, 1.495520e+03, 1.052626e+03],
           [0.000000e+00, 0.000000e+00, 1.000000e+00]]),
    'R': array([[ 0.6050639 ,

In [ ]:
cam_params = data['cam_param']
for key in cam_params.keys():
    c
    print(key, cam_params[key]['num_frame'])

KeyError: 'num_frame'